<a href="https://colab.research.google.com/github/Zain-Haider-ML/Regression-with-Transformer-Model/blob/main/Regression_with_Transformer_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#!/bin/bash
!kaggle datasets download nkitgupta/jigsaw-regression-based-data

Dataset URL: https://www.kaggle.com/datasets/nkitgupta/jigsaw-regression-based-data
License(s): unknown
 99% 2.65G/2.67G [00:36<00:00, 118MB/s] 
100% 2.67G/2.67G [00:36<00:00, 77.5MB/s]


In [ ]:
!unzip /content/jigsaw-regression-based-data.zip

Archive:  /content/jigsaw-regression-based-data.zip
  inflating: FastText-jigsaw-100D/Jigsaw-Fasttext-Word-Embeddings.bin  
  inflating: FastText-jigsaw-100D/Jigsaw-Fasttext-Word-Embeddings.bin.wv.vectors_ngrams.npy  
  inflating: FastText-jigsaw-256D/Jigsaw-Fasttext-Word-Embeddings-256D.bin  
  inflating: FastText-jigsaw-256D/Jigsaw-Fasttext-Word-Embeddings-256D.bin.syn1neg.npy  
  inflating: FastText-jigsaw-256D/Jigsaw-Fasttext-Word-Embeddings-256D.bin.wv.vectors_ngrams.npy  
  inflating: FastText-jigsaw-256D/Jigsaw-Fasttext-Word-Embeddings-256D.bin.wv.vectors_vocab.npy  
  inflating: train_data_version1.csv  
  inflating: train_data_version2.csv  
  inflating: train_data_version3.csv  


In [ ]:
!pip install datasets
!pip install evaluate

In [1]:
import pandas as pd
import numpy as np
import torch

In [2]:
ds = pd.read_csv('/content/train_data_version1.csv')
ds.head()

,text,y
0,oh why dear johnny boy part one again do you w...,2.08
1,savant13 is a fat shite,3.58
2,i am english i am i do not want to get blocked...,0.80
3,and why the fuck should i care g e t r i d o f...,4.58
4,yo dear sir james paul you a fruity ass nigga ...,2.08


In [3]:
min(ds['y']), max(ds['y'])

(0.0, 7.58)

In [4]:
ds.rename(columns={'y':'label'}, inplace=True)

In [5]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer
from sklearn.model_selection import train_test_split
from datasets import Dataset, load_dataset
from evaluate import load

In [6]:
import wandb
wandb.init(mode="disabled") # Changed: WANDB_MODE=disabled to mode="disabled"

In [7]:
train, test = train_test_split(ds[:500], test_size=0.1, random_state=42)

In [8]:
train = Dataset.from_pandas(train)
test = Dataset.from_pandas(test)

In [9]:
tokenizer = AutoTokenizer.from_pretrained('sentence-transformers/all-MiniLM-L6-v2')
model = AutoModelForSequenceClassification.from_pretrained('sentence-transformers/all-MiniLM-L6-v2', problem_type="regression", num_labels=1)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at sentence-transformers/all-MiniLM-L6-v2 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [10]:
def tokenize_function(examples):
    return tokenizer(examples['text'], padding="max_length", truncation=True)

In [11]:
tok_train = train.map(tokenize_function, batched=True)
tok_test = test.map(tokenize_function, batched=True)

Map:   0%|          | 0/450 [00:00<?, ? examples/s]

Map:   0%|          | 0/50 [00:00<?, ? examples/s]

In [12]:
epochs = 4
batch_size = 16

In [13]:
args = TrainingArguments(
    eval_strategy = "steps",
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs= epochs,
    output_dir='/content/',
    logging_steps = 5,
    report_to=None,
)

In [17]:
from sklearn.metrics import mean_squared_error
import numpy as np

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    mse = mean_squared_error(labels, predictions)  # No 'squared' argument here
    rmse = np.sqrt(mse)  # Manually compute the square root
    return {"rmse": rmse}


In [18]:
trainer = Trainer(
    model=model,
    args=args,
    train_dataset=tok_train,
    eval_dataset=tok_test,
    compute_metrics=compute_metrics,
    tokenizer=tokenizer,
)

<ipython-input-18-62f072208d99>:1: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [19]:
trainer.train()

Step,Training Loss,Validation Loss,Rmse
5,4.784700,7.569195,2.751217
10,6.185500,7.101828,2.664926
15,2.504000,6.559575,2.561167
20,4.141300,5.979219,2.445244
25,3.061500,5.407425,2.325387
30,2.795400,4.930495,2.220472
35,1.898400,4.544221,2.131718
40,1.767300,4.207607,2.051245
45,2.885000,3.934936,1.983667
50,2.531100,3.735786,1.932818


TrainOutput(global_step=116, training_loss=2.503121437183742, metrics={'train_runtime': 2988.0715, 'train_samples_per_second': 0.602, 'train_steps_per_second': 0.039, 'total_flos': 59696328499200.0, 'train_loss': 2.503121437183742, 'epoch': 4.0})

usage: git [--version] [--help] [-C <path>] [-c <name>=<value>]
           [--exec-path[=<path>]] [--html-path] [--man-path] [--info-path]
           [-p | --paginate | -P | --no-pager] [--no-replace-objects] [--bare]
           [--git-dir=<path>] [--work-tree=<path>] [--namespace=<name>]
           [--super-prefix=<path>] [--config-env=<name>=<envvar>]
           <command> [<args>]

These are common Git commands used in various situations:

start a working area (see also: git help tutorial)
   clone     Clone a repository into a new directory
   init      Create an empty Git repository or reinitialize an existing one

work on the current change (see also: git help everyday)
   add       Add file contents to the index
   mv        Move or rename a file, a directory, or a symlink
   restore   Restore working tree files
   rm        Remove files from the working tree and from the index

examine the history and state (see also: git help revisions)
   bisect    Use binary search to find th